<a href="https://colab.research.google.com/github/plaban1981/LLMs-Large-Language-Model/blob/main/GPT4all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q datasets loralib sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.7 MB/s eta 0:00:00


In [3]:
!pip uninstall transformers
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 11.8 MB/s eta 0:00:00


In [4]:
import torch
from peft import PeftModel,PeftConfig
from transformers import AutoModelForCausalLM,AutoTokenizer,GenerationConfig
import textwrap



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-cp78u4bfsncu --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=t

In [6]:
peft_model_id = "nomic-ai/gpt4all-lora"
config = PeftConfig.from_pretrained(peft_model_id)
#
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             return_dict=True,
                                             load_in_8bit=True,
                                             device_map="auto")
#
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Load the Lora Model

In [7]:
model = PeftModel.from_pretrained(model,peft_model_id)

## Define Generate Function

In [46]:
def gpt4all_generate(text):
  inputs = tokenizer(text,return_tensors="pt")
  input_ids = inputs["input_ids"].cuda()
  generation_config = GenerationConfig(temperature =0.7,
                                       top_p=0.95,
                                       repetition_pebalty=1.2,
                                       )
  print("Generating ....")
  generation_output = model.generate(input_ids =input_ids,
                                     generation_config = generation_config,
                                     output_scores=True,
                                     max_new_tokens=256,)
  wrapped_text = textwrap.fill(tokenizer.decode(generation_output[0]),width=100)
  print(wrapped_text)

In [13]:
%%time
PROMPT = """Below is an instrution that describes a task.Write a response that approximately completes the request.
###INSTRUCTION:
What are the differences between alpaca and sheep ?
###RESPONSE:"""
gpt4all_generate(PROMPT)

Generating ....
 Below is an instrution that describes a task.Write a response that approximately completes the
request. ###INSTRUCTION: What are the differences between alpaca and sheep ? ###RESPONSE: Alpacas
and sheep are both members of the camelid family, but they have some key differences. Alpacas are
smaller than sheep and have a more slender build. They also have a shorter, finer fleece that is
more resistant to pests and diseases. Alpacas are also known for their gentle nature and are often
used for their fiber, which is used to make clothing and other textiles. Sheep, on the other hand,
are larger and have a coarser, woolier fleece that is used for a variety of purposes, including
clothing, carpets, and blankets. Sheep are also known for their ability to graze on a variety of
grasses and other plants, making them more versatile than alpacas.
CPU times: user 37.8 s, sys: 0 ns, total: 37.8 s
Wall time: 37.8 s


In [14]:
%%time
PROMPT = """###INSTRUCTION:
What are the differences between alpaca and sheep ?
###RESPONSE:"""
gpt4all_generate(PROMPT)

Generating ....
 ###INSTRUCTION: What are the differences between alpaca and sheep ? ###RESPONSE: Alpacas and sheep
are both members of the camelid family, but they are not closely related. Alpacas are native to
South America, while sheep are native to Europe and Asia. Alpacas are smaller than sheep, with a
more slender build and a shorter, curly coat. They are also more docile and easier to handle than
sheep. Alpacas are primarily raised for their fiber, which is used to make clothing and other
textiles, while sheep are raised for their meat and wool.
CPU times: user 24.3 s, sys: 0 ns, total: 24.3 s
Wall time: 24.4 s


In [27]:
def gpt4all_generate(text):
  inputs = tokenizer(text,return_tensors="pt")
  input_ids = inputs["input_ids"].cuda()
  generation_config = GenerationConfig(temperature =0,
                                       top_p=0.95,
                                       repetition_penalty=1.2,
                                       )
  print("Generating ....")
  generation_output = model.generate(input_ids =input_ids,
                                     generation_config = generation_config,
                                     output_scores=True,
                                     max_new_tokens=1000,)
  #wrapped_text = textwrap.fill(tokenizer.decode(generation_output[0]),width=100)
  out = tokenizer.decode(generation_output[0])
  return out

In [23]:
%%time
PROMPT="""You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications. Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health.
###INSTRUCTION:
Please provide the details for all the medicines prescribed in a point-by-point manner:
1. Provide General information on the medications you listed in a point wise manner. 
2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications. 
3. Provide Proper dosage, administration, and storage of medications
4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.
5. Also correct the medication names if specified incorrectly.

The output should be generated in Markdown Format.Make sure details should be h2 tag & response to the details should be p tag

MEDICINES PRESCRIBED:
Medicines Prescribed:  
1. Glucobay 50 mg
2. Reclide 80 mg
3. Glycomet  25 mg
4. Glucobay 50 mg
5. Fourtiz B Z Plus
Allergic/ Intolerant:  
Metmorfin
###RESPONSE:"""
gpt4all_generate(PROMPT)

Generating ....
 You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will
provide real-time information about drugs, including side effects, interactions, and potential
contraindications. Simply tell me the name of your medication and I will do my best to provide you
with accurate and timely information to help you make informed decisions about your health.
###INSTRUCTION: Please provide the details for all the medicines prescribed in a point-by-point
manner: 1. Provide General information on the medications you listed in a point wise manner.  2.
Provide Real-time information about drugs, including side effects, interactions, and potential
contra-indications.  3. Provide Proper dosage, administration, and storage of medications 4. Suggest
Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.
5. Also correct the medication names if specified incorrectly.  The output should be generated in
Markdown For

## Trying the same without the Alpaca Prompt

In [28]:
%%time
PROMPT="""You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications.Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health.
\n
Please provide the details for all the medicines prescribed in a point-by-point manner:\n
1. Provide General information on the medications you listed in a point wise manner. \n
2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications.\n 
3. Provide Proper dosage, administration, and storage of medications. \n
4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.\n
5. Also correct the medication names if specified incorrectly.\n
\n
The output should be generated in Markdown Format.Make sure details should be h2 tag & response to the details should be p tag
\n
MEDICINES PRESCRIBED:\n
Medicines Prescribed:  
1. Glucobay 50 mg
2. Reclide 80 mg
3. Glycomet  25 mg
4. Glucobay 50 mg
5. Fourtiz B Z Plus
Allergic/ Intolerant:  
Metmorfin
###RESPONSE:\n
"""
out = gpt4all_generate(PROMPT)

Generating ....
CPU times: user 1min 15s, sys: 127 ms, total: 1min 15s
Wall time: 1min 15s


In [32]:
print(textwrap.fill(out,100))

 You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will
provide real-time information about drugs, including side effects, interactions, and potential
contraindications.Simply tell me the name of your medication and I will do my best to provide you
with accurate and timely information to help you make informed decisions about your health.   Please
provide the details for all the medicines prescribed in a point-by-point manner:  1. Provide General
information on the medications you listed in a point wise manner.   2. Provide Real-time information
about drugs, including side effects, interactions, and potential contra-indications.   3. Provide
Proper dosage, administration, and storage of medications.   4. Suggest Potential alternatives for
patients who are allergic or intolerant to specific prescriptions if any.  5. Also correct the
medication names if specified incorrectly.    The output should be generated in Markdown Format.Make
sure details 

In [33]:
! pip install -qU openai

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_shell.py:98 in system                       │
│                                                                                                  │
│    95 │   if pip_warn:                                                                           │
│    96 │     kwargs.update({'also_return_output': True})                                          │
│    97 │                                                                                          │
│ ❱  98 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│    99 │                                                                                          │
│   100 │   if pip_warn:                                                                           │
│   101 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:453 in _system_compat    │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/google/colab/_system_commands.py:167 in _run_command      │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [36]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## Using ChatGPT API

In [37]:
! pip install -qU openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 7.8 MB/s eta 0:00:00


In [40]:
import openai
from time import time,sleep
openai.api_key = "<<your api key here>>"

In [42]:
def predict_function(prompt):
    #
    messages = [{"role":"system",
             "content":"Your are a helpful assistant.",
             },
            ]
    #
    messages.append({"role":"user","content":prompt})
    print(messages)
    #
    max_retry = 5
    retry = 0
    #
    
    while True:
        try:
            chat = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages = messages,
                                        )
            reply = chat.choices[0].message.content 
            return reply
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "Accessing the Completion service error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)

In [43]:
%%time
prompt = """You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications. Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health. Please provide the details in a point-by-point manner:
1. Provide General information on the medications you listed in a point wise manner. 
2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications. 
3. Provide Proper dosage, administration, and storage of medications
4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.
5. Also correct the medication names if specified incorrectly.

The output should be generated in Markdown Format.Make sure details should be h2 tag & response to the details should be p tag

MEDICINES PRESCRIBED:
Medicines Prescribed:  
1. Glucobay 50 mg
2. Reclide 80 mg
3. Glycomet  25 mg
4. Glucobay 50 mg
5. Fourtiz B Z Plus
Allergic/ Intolerant:  
Metmorfin

"""
predict_function(prompt)

[{'role': 'system', 'content': 'Your are a helpful assistant.'}, {'role': 'user', 'content': 'You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications. Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health. Please provide the details in a point-by-point manner:\n1. Provide General information on the medications you listed in a point wise manner. \n2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications. \n3. Provide Proper dosage, administration, and storage of medications\n4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.\n5. Also correct the medication names if specified incorrectly.\n\nThe output 

'## Information on the prescribed medications\n\n1. **Glucobay 50 mg**: This medication contains the active ingredient Acarbose and is used to treat type 2 diabetes. It helps to lower high blood sugar levels after meals by delaying the digestion and absorption of carbohydrates in the intestine.\n2. **Reclide 80 mg**: This medication contains the active ingredient Gliclazide and is used to treat type 2 diabetes. It helps to lower high blood sugar levels by stimulating the pancreas to produce more insulin.\n3. **Glycomet 25 mg**: This medication contains the active ingredient Metformin and is used to treat type 2 diabetes. It helps to lower high blood sugar levels by reducing the amount of glucose produced in the liver and increasing the sensitivity of muscle cells to insulin.\n4. **Glucobay 50 mg**: This medication contains the active ingredient Acarbose and is used to treat type 2 diabetes. It helps to lower high blood sugar levels after meals by delaying the digestion and absorption o

In [44]:
print(predict_function(prompt))

[{'role': 'system', 'content': 'Your are a helpful assistant.'}, {'role': 'user', 'content': 'You are a helpful healthcare assistant. Ask me anything about your medication regimen and I will provide real-time information about drugs, including side effects, interactions, and potential contraindications. Simply tell me the name of your medication and I will do my best to provide you with accurate and timely information to help you make informed decisions about your health. Please provide the details in a point-by-point manner:\n1. Provide General information on the medications you listed in a point wise manner. \n2. Provide Real-time information about drugs, including side effects, interactions, and potential contra-indications. \n3. Provide Proper dosage, administration, and storage of medications\n4. Suggest Potential alternatives for patients who are allergic or intolerant to specific prescriptions if any.\n5. Also correct the medication names if specified incorrectly.\n\nThe output 

## Comaprison of response from chatGPT vs GPT4all

* response took 38 seconds an output is more structured in chatgpt
* response took 1min 15 seconds for gpt4all for the same prompt and output is not very structured.
* also ChapGPT understands what is markdown format while gpt4all was unable to interpret it.
* also chatgpt generated the response within the default max_tokens i.e 256 limits but the max_new_tokens limit for gpt4all was set to 1000 without which the result was getting truncated.

##Code Generation - ChatGPT

In [57]:
%%time
prompt = """You are a helpful Coding assistant. Ask me anything about coding logic in any programming language and I will provide the code for the logic requested:
Write a python code to detect if a give string is palindrome or not.  
CODE GENERATED:

"""
print(predict_function(prompt))

[{'role': 'system', 'content': 'Your are a helpful assistant.'}, {'role': 'user', 'content': 'You are a helpful Coding assistant. Ask me anything about coding logic in any programming language and I will provide the code for the logic requested:\nWrite a python code to detect if a give string is palindrome or not.  \nCODE GENERATED:\n\n'}]
Here's a Python code to check if a given string is a palindrome or not:

```
def check_palindrome(string):
  # Convert all letters to lowercase
  string = string.lower()
  # Reverse the string
  reversed_string = string[::-1]
  # Compare the reversed string to the original string
  if string == reversed_string:
    return True
  else:
    return False
  
# Example usage
print(check_palindrome("racecar"))  # True
print(check_palindrome("hello"))  # False
``` 

In this code, we define a function called `check_palindrome` that takes a `string` parameter. We first convert all the letters of the string to lowercase by using the `lower()` method. We then r

## Code generation -GPT4all

In [58]:
%%time
PROMPT = """You are a helpful Coding assistant. Ask me anything about coding logic in any programming language and I will provide the code for the logic requested:
Write a python code to detect if a give string is palindrome or not.  
CODE GENERATED:

"""
gpt4all_generate(PROMPT)

Generating ....
 You are a helpful Coding assistant. Ask me anything about coding logic in any programming language
and I will provide the code for the logic requested: Write a python code to detect if a give string
is palindrome or not.   CODE GENERATED:  ``` def is_palindrome(string):     if string ==
string[::-1]:         return True     else:         return False ```
CPU times: user 7.7 s, sys: 19.5 ms, total: 7.72 s
Wall time: 9.44 s


* the GPT4all took more CPU time compared to ChatGPT

In [59]:
def is_palindrome(string):
  if string ==string[::-1]:
    return True
  else:         
    return False

In [60]:
is_palindrome('MADAM')

True